## ce script n'est pas intéressant car il y a déjà un librairie de traitement d'image

## encadrer, tirer les partie interesants dans l'image et les enregister 

In [ ]:
import cv2
import numpy as np
import os , sys            
workpath = sys.path[0]

In [ ]:
"""
path_img = f'{workpath}\datasheet\mmClasses-DataBase-IMG\mmClasses-DataBase-IMG\data_64_equilibre'
L = os.listdir(path_img)
img = f'{path_img}\{L[500]}'
img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
"""
path_img = f'{workpath}\qq_img_relle'
L = os.listdir(path_img)
img = f'{path_img}\{L[0]}'
img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)

In [ ]:
print(path_img)
print(type(path_img))

In [ ]:
BGR_img =  cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
HSV_img = cv2.cvtColor(BGR_img, cv2.COLOR_BGR2HSV)

In [ ]:
cv2.namedWindow("HSV_Image",0)
cv2.resizeWindow("HSV_Image",300,300)
cv2.imshow("HSV_Image", HSV_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
print(HSV_img.shape)

In [ ]:
print(HSV_img[600][2000])

In [ ]:
def get_mask_image(hsv_image, lower, upper):
    """
    obtenir un image binaire: qui n'a que 2 valeur 0 et 255
    les pixels qui ont une valeur inferieur que lower et surperieur que upper obtiennent 0, les autre 255 
    
    j'ai pas encore compris cette fonction
    """
    mask_image = cv2.inRange(hsv_image, lower, upper)  # obtenir un image binaire
    
    cv2.namedWindow("Mask Image",0)
    cv2.resizeWindow("Mask Image",300,300)
    cv2.imshow("Mask Image", mask_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print(mask_image.shape)
    return mask_image

In [ ]:
lower = np.array([0, 0, 10])
upper = np.array([0, 0, 180])
mask_image = get_mask_image(HSV_img, lower, upper)  

In [ ]:
def get_morphology_image(mask_image):
    """
     Traitement morphologique des images binarisées - éliminer les effets du bruit
      1) Commencer par un processus d'inflation
         Le gonflage est l'utilisation d'un algorithme pour étendre les bords d'une image. L'effet est de remplir les bords de la cible ou les piqûres internes et de supprimer les petits trous.
      2) Corrosion
         L'érosion : l'érosion réduit les limites d'un objet, principalement pour éliminer le bruit blanc, mais aussi pour déconnecter des objets qui sont connectés.

       Traduit avec www.DeepL.com/Translator (version gratuite)
    """

    kernel = np.ones((5, 5), np.uint8)
    dialationIamge = cv2.dilate(mask_image, kernel, iterations=1)  # 膨胀处理
    eroded_iamge = cv2.erode(dialationIamge, kernel, iterations=1)  # 腐蚀处理

    cv2.namedWindow("Eroded Image",0)
    cv2.resizeWindow("Eroded Image",300,300)
    cv2.imshow("Eroded Image", eroded_iamge)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
      # 显示形态学处理过后的图片
    return eroded_iamge


In [ ]:
eroded_image = get_morphology_image(mask_image)

In [ ]:
def get_contours(eroded_iamge):
    """
    Obtenir le contour de l'image et dessiner le contour sur l'image originale.
       L'entrée est une image binaire traitée morphologiquement avec le noir comme arrière-plan et le blanc comme cible.
       cv2.RETR_EXTERNAL détecte uniquement les contours extérieurs
       cv2.CHAIN_APPROX_SIMPLE
    """
 
    contours, hierarchy = cv2.findContours(eroded_iamge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # obtenir un contour pour chaque partie de image

    return contours

In [ ]:
def get_and_save_areas(contours, original_image):
    """
       couper les partie interessant dans les contour dans l'image initial
    """
    min_area = 300  # filter les petit espace bruité dans l'image initial
    index = 0  # nombre de partie interessant
    for numcontours, contour in enumerate(contours):
        image_area = cv2.contourArea(contour)
        if image_area > min_area:
            x, y, w, h = cv2.boundingRect(contour)
            module_image = original_image[y:y + h, x: x + w]
            index = index + 1
            path_storage = path_img + '/' + str(x) + '_' + str(y) + '_' + str(w) + '_' + str(h) + '_' + str(image_area) + '_' + str(index) + '.jpg'
            print(path_storage)
            cv2.imwrite(path_storage, module_image)  # sauvgarder les image dans le path_storage et les nommer par module_image

    return index

In [ ]:
index = get_and_save_areas(get_contours(eroded_image), img)